In [ ]:
import pandas as pd
import math
import numpy as np
import os
import requests
import urllib
import urllib3
from census import Census
from us import states
import multiprocessing
from multiprocessing import Pool
import time 


In [ ]:
###csv file we got from DT
store_attr=pd.read_csv("/working/discount_tire/data/store_attributes.csv")

In [ ]:
store_attr["close_date"] = np.where(store_attr['store_sales_close_date']=='9999-12-31', pd.datetime.now().strftime("%Y-%m-%d"), store_attr["store_sales_close_date"])
store_attr['store_open_date']=pd.to_datetime(store_attr['store_open_date'])
store_attr['close_date']=pd.to_datetime(store_attr['close_date'])
store_attr['store_age'] = store_attr['close_date'] - store_attr['store_open_date']
store_attr['store_age'] = store_attr['store_age'] /  np.timedelta64(1, 'Y')
del store_attr['close_date']
#store_attr

In [ ]:
### getting distance from closest store in km 
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

data_new = store_attr[['store_code','store_latitude_degree', 'store_longitude_degree']].drop_duplicates()
data_new['key'] = 1
final_data = data_new.merge(data_new, how = 'left', on = ['key'])
final_data['distance_in_km'] = haversine_np(final_data['store_longitude_degree_x'],final_data['store_latitude_degree_x'],final_data['store_longitude_degree_y'],final_data['store_latitude_degree_y'])

final_data=final_data[final_data['store_code_x']!=final_data['store_code_y']]
final_data=final_data.groupby('store_code_x')['distance_in_km'].min()
final_data=pd.DataFrame(final_data)
final_data=final_data.reset_index()
final_data.rename(columns={'store_code_x':'store_code','distance_in_km':'distance_to_nearest_store_km'}, inplace=True)
store_attr=pd.merge(store_attr,final_data, on=['store_code'], how='left')
#store_attr

In [ ]:
#historical weather info for all the stores
# This file is generated from hive based on 30 years of data
store_hist_weather_info=pd.read_csv("/working/discount_tire/faisal/feature_addition/store_hist_weather_info.csv")
store_hist_weather_info = store_hist_weather_info.drop(store_hist_weather_info.columns[[0]], axis=1)
store_hist_weather_info=store_hist_weather_info[['store_code','avg_temp','avg_pcp','avg_acc_snow','max_temp','max_pcp','max_acc_snow',
                                                 'min_temp','min_pcp','min_acc_snow','stddev_temp','stddev_pcp','stddev_acc_snow']]
store_attr=pd.merge(store_attr, store_hist_weather_info, on=['store_code'], how='left')
#store_attr

In [ ]:
#additional_store_characterstics
## This file is from DT
lot_and_building_characteristics=pd.read_csv("/working/discount_tire/faisal/feature_addition/lot_and_building_characteristics.csv")
lot_and_building_characteristics.rename(columns={'Store ID': 'store_code','Building Size (SF)':'building_size_sqft','Land Size (SF)':'land_size_sqft','# of Bays':'no_of_bays','# of Parking Stalls':'no_of_parking_stalls','Shared Parking':'shared_parking','Store Format':'store_format'}, inplace=True)
lot_and_building_characteristics=lot_and_building_characteristics[["store_code","building_size_sqft","land_size_sqft","no_of_bays","no_of_parking_stalls"]]
lot_and_building_characteristics['no_of_parking_stalls'] = np.where(lot_and_building_characteristics['no_of_parking_stalls'] =='40+', 45, lot_and_building_characteristics['no_of_parking_stalls'])

store_attr=pd.merge(store_attr, lot_and_building_characteristics, on=['store_code'], how='left')
#store_attr

In [ ]:
# supporting functions to request json response from API and create the elevation column
def make_remote_request(url: str, params: dict):
    """
    Makes the remote request
    Continues making attempts until it succeeds
    """

    count = 1
    while True:
        try:
            response = requests.get((url + urllib.parse.urlencode(params)))
        except (OSError, urllib3.exceptions.ProtocolError) as error:
            print('\n')
            print('*' * 20, 'Error Occured', '*' * 20)
            print('Number of tries:{}'.format(count))
            print('URL: {}'.format(url))
            print(error)
            print('\n')
            count += 1
            continue
        break

    return response


def elevation_function(x):
    url = 'https://nationalmap.gov/epqs/pqs.php?'
    params = {'x': x['store_longitude_degree'],
              'y': x['store_latitude_degree'],
              'units': 'Meters',
              'output': 'json'}
    result = make_remote_request(url, params)
    return result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation']


store_attr['elevation_meter'] = store_attr.apply(elevation_function, axis=1)
#store_attr

In [ ]:
####key set up steps
'''
Step 1 - Go to https://www.census.gov/data/developers/guidance/api-user-guide.html

Step 2 - Click on "Request a Key"

Step 3 - Enter your organisation name and email address 

Step 4 - You will receive the API Key in email provided.

'''

####Details
'''
Data source: api.census.gov (American Community Survey – 2018) ​

Information available at zip code level – mapped store to corresponding zip code​

'census' Package used to call values from the API​

Added Features on zip code level: ​

Population/male population/female population​

Aggregate number of vehicles (cars, trucks or vans)​

median income in dollars​

No. of housing units ​

Individuals owning 0/1/2/3/4/5 or more number of vehicles​
'''

In [ ]:
key = 'f34e78b85a544c493c9dcdfe4c00a893238052fa' #unique key required for API call  To do link to doc to have more understanding
c = Census(key) 

def add_features(df):
    
    def func(pin, code, dummy):
        pin = pin.split('-')[0]
        try:
            v = c.acs5.zipcode(code, pin)[0][code]
            return v
        except:
            return None
    
    
    df['total_population'] = df['store_postal_code'].apply(func, args=('B01001_002E',None))
    
    df['male_population'] = df['store_postal_code'].apply(func, args=('B01001_001E',None))

    df['female_population'] = df['store_postal_code'].apply(func, args=('B01001_002E',None))

    df['housing_unit'] = df['store_postal_code'].apply(func, args=('B25001_001E',None))
    
    df['aggregate_vehicles'] = df['store_postal_code'].apply(func, args=('B08015_001E',None))
    
    df['median income'] = df['store_postal_code'].apply(func, args=('B06011_001E',None))   # median income
    
    df['individuals_with_no_vehicle'] = df['store_postal_code'].apply(func, args=('B08014_002E',None))  # individuals with no vehicle

    df['individuals_with_one_vehicle'] = df['store_postal_code'].apply(func, args=('B08014_003E',None))  # individuals with 1 vehicle

    df['individuals_with_two_vehicle'] = df['store_postal_code'].apply(func, args=('B08014_004E',None))  # individuals with 2 vehicles

    df['individuals_with_three_vehicle'] = df['store_postal_code'].apply(func, args=('B08014_005E',None))  # individuals with 3 vehicles

    df['individuals_with_four_vehicle'] = df['store_postal_code'].apply(func, args=('B08014_006E',None))  # individuals with 4 vehicles

    df['individuals_with_five_or_more_vehicle'] = df['store_postal_code'].apply(func, args=('B08014_007E',None))  # individuals with 5 or more vehicles

    df['individuals_total'] = df['store_postal_code'].apply(func, args=('B08014_001E',None))  # Total vehicles data available

    

    return df

In [ ]:
def parallelize_dataframe(df, function):
    n_cores=(multiprocessing.cpu_count()) - 8
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(function, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
df=store_attr[['store_code','store_postal_code']]
#df=df.head(5)
start = time.time()
final_census = parallelize_dataframe(df, add_features)
print(time.time()-start)
#final_census
del final_census['store_postal_code']

In [ ]:
store_attr=pd.merge(store_attr, final_census, on=['store_code'], how='left')
#store_attr

In [ ]:
store_attr.to_csv('final_store_attributes.csv', index=False)